In [1]:
import datetime as dt
import pandas as pd 
import numpy as np
import os
import shutil
from IPython.display import HTML, display


# Files extension 
EXTENSION_XLSX = ".xlsx"

# Project directories
DATA_SOURCE_DIRECTORY_NAME = "Data source"
REPORTS_DIRECTORY_NAME = "Reports"  
VMK_DIRECTORY_NAME = "ВМК"
VMK2_DIRECTORY_NAME = "ВМК2"
ROSTELECOM_DIRECTORY_NAME = "Ростелеком"

# Basic file names for data 
VMK_FILE_NAME = "Отчет по вызовам домена 997267.34.rt.ru"
VMK2_FILE_NAME = "Отчет по вызовам домена 553009.34.rt.ru"
ROSTELECOM_FILE_NAME = "Отчет по вызовам в домене 997267.34.rt.ru"

# Group names
VMK_GROUP = "ВМК"
VMK2_GROUP = "ВМК2"
ROSTELECOM_GROUP = "Ростелеком"

# Project data paths
# Local paths
PATH_DATA_SOURCE = os.path.join(".\\", DATA_SOURCE_DIRECTORY_NAME)
PATH_REPORTS = os.path.join(".\\", REPORTS_DIRECTORY_NAME)
DOWNLOADS_PATH = f"C:\\Users\\{os.getlogin( )}\\Downloads"

# Network data paths
ROOT_PATHS_NETWORK_STORAGE = ["Z:/ОСЦ/PUBLIC/Недозвоны/", "\\\db1\FS\\ОСЦ\\PUBLIC\\Недозвоны\\"]

# Стратегия загрузки файлов в хранилище проекта
# 1.	Загрузка с предварительной проверкой на актуальность
IS_STRATEGY_LOAD_FILES_BY_CHECK = True
# 2.	Загрузка без проверок 
IS_STRATEGY_LOAD_FILES_WITHOUT_CHECK = False


In [2]:
def create_all_possible_paths_project_storage(dict_all_paths_project_storage):
    dict_all_paths_project_storage = creat_paths_for_data(PATH_DATA_SOURCE, dict_all_paths_project_storage)
    dict_all_paths_project_storage = creat_paths_for_data(ROOT_PATHS_NETWORK_STORAGE[0], dict_all_paths_project_storage)
    dict_all_paths_project_storage = creat_paths_for_data(ROOT_PATHS_NETWORK_STORAGE[1], dict_all_paths_project_storage)
    return dict_all_paths_project_storage


def creat_paths_for_data(root_path_for_data, path_list):
    if os.access(root_path_for_data, os.F_OK):
        path_list["path_list_vmk"].append(os.path.join(root_path_for_data, VMK_DIRECTORY_NAME))
        path_list["path_list_vmk2"].append(os.path.join(root_path_for_data, VMK2_DIRECTORY_NAME))
        path_list["path_list_rostelecom"].append(os.path.join(root_path_for_data, ROSTELECOM_DIRECTORY_NAME))
    return path_list


def load_one_table_for_insert(path_load_file, is_VMK=False): 
    if os.access(path_load_file, os.F_OK):
        df = pd.read_excel(path_load_file)
        if is_VMK:        
            data_copy_from_site = True
            # Проверка на скачен файл ли сайта
            if 'Номер телефона' not in df.columns:
                list_index = list()
                for i in range(len(df)):
                    list_index.append(i)  
                    if 'Номер телефона' in tuple(df.iloc[i]):
                        df.columns = df.iloc[i]
                        df.columns.name=None
                        df = df.drop(index=list_index)
                        df.reset_index(inplace=True, drop=True)
                        
                        data_copy_from_site = False
                        break
            
            df['Время звонка'] = df['Время звонка'].astype('str')
            df['Время ожидания'] = pd.to_timedelta(df['Время ожидания'].astype('str'))
            df['Время разговора'] = pd.to_timedelta(df['Время разговора'].astype('str'))
            
            if data_copy_from_site:
                df = df.groupby((df[['Номер телефона']].isnull()==False).all(1).cumsum()).agg({'Время звонка': ' '.join, 'Номер телефона': 'first', 'Статус звонка': 'first',
                            'Время ожидания': 'first', 'Время разговора': 'first',
                            'Группа': 'first', 'Оператор': 'first'})
                df['Время звонка'] = df['Время звонка'].apply(lambda x: dt.datetime.strptime(x, '%H:%M:%S %Y-%m-%d 00:00:00'))
            else:
                df['Время звонка'] = df['Время звонка'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        else:
            df["Дата вызова"] = df["Дата вызова"].astype('datetime64')
            df["Время вызова"] = df["Время вызова"].astype('datetime64')         
            # print(df["Время вызова"].max().time())
            # print(df["Дата вызова"].max().date())
            # print(df.info())  
        return df

# print(f'Добавление файла: {file_name} в проект не выполнено! В файле отсутствуют данные. Путь: {DOWNLOADS_PATH}, группа: {name_group_files}')   


def insert_data_report_for_insert_files(dict_report_for_insert_files, name_insert_file, directory_insert_file, min_date_call_insert_file, max_date_call_insert_file, max_time_call_insert_file, 
                                        min_time_call_insert_file, name_update_file, directory_update_file, min_time_call_update_file, max_time_call_update_file, status):
    dict_report_for_insert_files['Имя вставляемого файла'].append(name_insert_file)
    dict_report_for_insert_files['Директория вставляемого файла'].append(directory_insert_file)
    dict_report_for_insert_files['Минимальная дата во вставляемом файле'].append(min_date_call_insert_file)
    dict_report_for_insert_files['Максимальная дата  во вставляемом файле'].append(max_date_call_insert_file)
    dict_report_for_insert_files['Минимальное время звонка во вставляемом файле'].append(min_time_call_insert_file)
    dict_report_for_insert_files['Максимальное время звонка во вставляемом файле'].append(max_time_call_insert_file)
    dict_report_for_insert_files['Имя обновляемого/добавляемого файла'].append(name_update_file)
    dict_report_for_insert_files['Директория обновляемого/добавляемого файла'].append(directory_update_file)
    dict_report_for_insert_files['Минимальное время звонка в обновляемом/добавляемом файле'].append(min_time_call_update_file)
    dict_report_for_insert_files['Максимальное время звонка в обновляемом/добавляемом файле'].append(max_time_call_update_file)
    dict_report_for_insert_files['Статус'].append(status)
    return dict_report_for_insert_files


def copy_file(path_insert_file, path_storage_file):
    shutil.copyfile(path_insert_file, path_storage_file)

 
def insert_data_project_storage(list_files_for_insert, list_paths_data_group, name_group_files, is_VMK=False):
    report_for_insert_files = {'Имя вставляемого файла': [], 'Директория вставляемого файла': [], 'Минимальная дата во вставляемом файле': [], 'Максимальная дата  во вставляемом файле': [],  
                                    'Минимальное время звонка во вставляемом файле': [], 'Максимальное время звонка во вставляемом файле': [], 'Имя обновляемого/добавляемого файла': [], 
                                    'Директория обновляемого/добавляемого файла': [], 'Минимальное время звонка в обновляемом/добавляемом файле': [], 
                                    'Максимальное время звонка в обновляемом/добавляемом файле': [], 'Статус': []}
    
    for file_name_to_insert in list_files_for_insert:
        file_path_insert_tmp = os.path.join(DOWNLOADS_PATH, file_name_to_insert)
        table_insert = load_one_table_for_insert(path_load_file=file_path_insert_tmp, is_VMK=is_VMK)
        
        # TODO: Устранить повторение кода
        max_datetime_table_insert = table_insert["Время звонка"].max() if is_VMK else None
        min_datetime_table_insert = table_insert["Время звонка"].min() if is_VMK else None

        max_date_table_insert = max_datetime_table_insert.date() if is_VMK else table_insert['Дата вызова'].max().date()
        min_date_table_insert = min_datetime_table_insert.date() if is_VMK else table_insert['Дата вызова'].min().date()

        max_time_call_table_insert = max_datetime_table_insert.time() if is_VMK else table_insert['Время вызова'].max().time()
        min_time_call_table_insert = min_datetime_table_insert.time() if is_VMK else table_insert['Время вызова'].min().time()
          
        if max_date_table_insert != min_date_table_insert:
            report_for_insert_files = insert_data_report_for_insert_files(dict_report_for_insert_files=report_for_insert_files, name_insert_file=file_name_to_insert, directory_insert_file=DOWNLOADS_PATH, 
                                                                        min_date_call_insert_file=min_date_table_insert, max_date_call_insert_file=max_date_table_insert, min_time_call_insert_file='-',
                                                                        max_time_call_insert_file='-', name_update_file='-', directory_update_file='-',  min_time_call_update_file='-', max_time_call_update_file='-', 
                                                                        status='Добавление файла не выполнено! Обнаружено несколько дат во вставляемом файле')
            continue

        for path_data_one_group in list_paths_data_group:
            
            vmk_file_name_project_storage =  (max_date_table_insert + dt.timedelta(days=1)).strftime("%Y-%m-%d") + EXTENSION_XLSX
            rostelecom_file_name_project_storage = max_date_table_insert.strftime("%Y-%m-%d") + EXTENSION_XLSX
            file_path_project_storage_tmp = os.path.join(path_data_one_group, vmk_file_name_project_storage) if is_VMK else os.path.join(path_data_one_group, rostelecom_file_name_project_storage)
        
            if IS_STRATEGY_LOAD_FILES_BY_CHECK:
                table_storage_project = load_one_table_for_insert(path_load_file=file_path_project_storage_tmp, is_VMK=is_VMK)
                
                if table_storage_project is not None:

                    # TODO: Устранить повторение кода
                    max_datetime_storage_project = table_storage_project["Время звонка"].max() if is_VMK else None
                    min_datetime_storage_project = table_storage_project["Время звонка"].min() if is_VMK else None  
                    max_time_call_table_storage_project = max_datetime_storage_project.time() if is_VMK else table_storage_project['Время вызова'].max().time()
                    min_time_call_table_storage_project = min_datetime_storage_project.time() if is_VMK else table_storage_project['Время вызова'].min().time()
                
                    is_invalid_max_time_call_table_insert = False
                    is_invalid_min_time_call_table_insert = False

                    if max_time_call_table_insert <= max_time_call_table_storage_project:
                        is_invalid_max_time_call_table_insert = True
                                                                  
                    if min_time_call_table_insert >= min_time_call_table_storage_project:
                         is_invalid_min_time_call_table_insert = True
                    
                    
                    if (not is_invalid_max_time_call_table_insert and not is_invalid_min_time_call_table_insert) or (is_invalid_max_time_call_table_insert and not is_invalid_min_time_call_table_insert) or (not is_invalid_max_time_call_table_insert and is_invalid_min_time_call_table_insert):
                        copy_file(path_insert_file=file_path_insert_tmp, path_storage_file=file_path_project_storage_tmp)
                        status_to_insert = 'Добавление файла выполнено!'
                    elif is_invalid_max_time_call_table_insert and is_invalid_min_time_call_table_insert:
                        status_to_insert = 'Добавление файла не выполнено! 1. Максимальное время во вставляемом файле меньше или равно в обновляемом файле. 2. Минимальное время во вставляемом файле больше или равно в обновляемом файле.'
                    # elif is_invalid_max_time_call_table_insert:
                    #     status_to_insert = 'Добавление файла не выполнено! Максимальное время во вставляемом файле меньше или равно в обновляемом файле'
                    # elif is_invalid_min_time_call_table_insert:
                    #     status_to_insert = 'Добавление файла не выполнено! Минимальное время во вставляемом файле больше или равно в обновляемом файле'
                

                    report_for_insert_files = insert_data_report_for_insert_files(dict_report_for_insert_files=report_for_insert_files, name_insert_file=file_name_to_insert, directory_insert_file=DOWNLOADS_PATH, 
                                                                        min_date_call_insert_file=min_date_table_insert, max_date_call_insert_file=max_date_table_insert, min_time_call_insert_file=min_time_call_table_insert,
                                                                        max_time_call_insert_file=max_time_call_table_insert, name_update_file=vmk_file_name_project_storage if is_VMK else rostelecom_file_name_project_storage, 
                                                                        directory_update_file=path_data_one_group, min_time_call_update_file=min_time_call_table_storage_project, max_time_call_update_file=max_time_call_table_storage_project, 
                                                                        status=status_to_insert)   
                else:
                    copy_file(path_insert_file=file_path_insert_tmp, path_storage_file=file_path_project_storage_tmp)
                    report_for_insert_files = insert_data_report_for_insert_files(dict_report_for_insert_files=report_for_insert_files, name_insert_file=file_name_to_insert, directory_insert_file=DOWNLOADS_PATH, 
                                                                        min_date_call_insert_file=min_date_table_insert, max_date_call_insert_file=max_date_table_insert, min_time_call_insert_file=min_time_call_table_insert,
                                                                        max_time_call_insert_file=max_time_call_table_insert, name_update_file=vmk_file_name_project_storage if is_VMK else rostelecom_file_name_project_storage, 
                                                                        directory_update_file=path_data_one_group, min_time_call_update_file=min_time_call_table_insert, max_time_call_update_file=max_time_call_table_insert, 
                                                                        status='Добавление файла выполнено!')
            
            elif IS_STRATEGY_LOAD_FILES_WITHOUT_CHECK:
                copy_file(path_insert_file=file_path_insert_tmp, path_storage_file=file_path_project_storage_tmp)
                report_for_insert_files = insert_data_report_for_insert_files(dict_report_for_insert_files=report_for_insert_files, name_insert_file=file_name_to_insert, directory_insert_file=DOWNLOADS_PATH, 
                                                                        min_date_call_insert_file=min_date_table_insert, max_date_call_insert_file=max_date_table_insert, min_time_call_insert_file=min_time_call_table_insert, 
                                                                        max_time_call_insert_file=max_time_call_table_insert, name_update_file=vmk_file_name_project_storage if is_VMK else rostelecom_file_name_project_storage, 
                                                                        directory_update_file=path_data_one_group,  min_time_call_update_file=min_time_call_table_insert, max_time_call_update_file=max_time_call_table_insert,
                                                                        status='Добавление файла выполнено!')
                
    return report_for_insert_files
    

# Создание списка имен файлов по группам, которые подлежат добавлению в проект 
file_list_dir_download = os.listdir(DOWNLOADS_PATH)
files_to_insert_dir_download = {"ВМК": [file for file in file_list_dir_download if VMK_FILE_NAME in file and EXTENSION_XLSX in file],
                                "ВМК2":[file for file in file_list_dir_download if VMK2_FILE_NAME in file and EXTENSION_XLSX in file],
                                "Ростелеком": [file for file in file_list_dir_download if ROSTELECOM_FILE_NAME in file and EXTENSION_XLSX in file]}

# Создание словаря всех путей к данным
dictionary_all_paths_project_storage = {"path_list_vmk": [], "path_list_vmk2": [], "path_list_rostelecom": []}
dictionary_all_paths_project_storage = create_all_possible_paths_project_storage(dictionary_all_paths_project_storage)

# Обновление директорий новыми данными
report_for_insert_files_VMK = insert_data_project_storage(list_files_for_insert=files_to_insert_dir_download["ВМК"], list_paths_data_group=dictionary_all_paths_project_storage['path_list_vmk'], 
                            name_group_files=VMK_GROUP, is_VMK=True)
report_for_insert_files_VMK2 = insert_data_project_storage(list_files_for_insert=files_to_insert_dir_download["ВМК2"], list_paths_data_group=dictionary_all_paths_project_storage['path_list_vmk2'], 
                            name_group_files=VMK2_GROUP, is_VMK=True)
report_for_insert_files_Rostelecom = insert_data_project_storage(list_files_for_insert=files_to_insert_dir_download["Ростелеком"], list_paths_data_group=dictionary_all_paths_project_storage['path_list_rostelecom'], 
                            name_group_files=ROSTELECOM_GROUP)


df_report_for_insert_files_VMK = None
df_report_for_insert_files_VMK2 = None
df_report_for_insert_files_Rostelecom = None
df_report_for_insert_files_all_groups = pd.DataFrame(columns=['Имя вставляемого файла', 'Директория вставляемого файла', 'Минимальная дата во вставляемом файле', 'Максимальная дата  во вставляемом файле', 
                                    'Минимальное время звонка во вставляемом файле', 'Максимальное время звонка во вставляемом файле', 'Имя обновляемого/добавляемого файла', 'Директория обновляемого/добавляемого файла', 
                                    'Минимальное время звонка в обновляемом/добавляемом файле', 'Максимальное время звонка в обновляемом/добавляемом файле', 'Статус'])

if len(report_for_insert_files_VMK['Имя вставляемого файла']) != 0:
    df_report_for_insert_files_VMK = pd.DataFrame(report_for_insert_files_VMK)

if len(report_for_insert_files_VMK2['Имя вставляемого файла']) != 0:
    df_report_for_insert_files_VMK2 = pd.DataFrame(report_for_insert_files_VMK2)    

if len(report_for_insert_files_Rostelecom['Имя вставляемого файла']) != 0:
    df_report_for_insert_files_Rostelecom = pd.DataFrame(report_for_insert_files_Rostelecom)    
    

df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_VMK,  ignore_index=True)
df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_VMK2,  ignore_index=True)
df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_Rostelecom, ignore_index=True)

C:\Users\AdminOS\AppData\Local\Temp\ipykernel_9236\450049872.py:202: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_VMK,  ignore_index=True)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_9236\450049872.py:203: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_VMK2,  ignore_index=True)
C:\Users\AdminOS\AppData\Local\Temp\ipykernel_9236\450049872.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_report_for_insert_files_all_groups = df_report_for_insert_files_all_groups.append(df_report_for_insert_files_R

### Отчет о добавлении файлов в проект

In [3]:
if IS_STRATEGY_LOAD_FILES_BY_CHECK:
    print('Стратегия загрузки файлов в хранилище проекта: загрузка с предварительной проверкой на актуальность')
elif IS_STRATEGY_LOAD_FILES_WITHOUT_CHECK:
    print('Стратегия загрузки файлов в хранилище проекта: загрузка без проверок')
else:
    print('Стратегия загрузки файлов в хранилище проекта: не выбрана ')

if IS_STRATEGY_LOAD_FILES_BY_CHECK or IS_STRATEGY_LOAD_FILES_WITHOUT_CHECK:
    if len(df_report_for_insert_files_all_groups) == 0:
        print(f'ВНИМАНИЕ!\nВставляемые файлы в директории: {DOWNLOADS_PATH} не обнаружены!')
    else:
        display(HTML(df_report_for_insert_files_all_groups.to_html()))
        df_report_for_insert_files_all_groups.to_excel(os.path.join(PATH_REPORTS,"Отчет о добавлении файлов в проект.xlsx"), index=False)

Стратегия загрузки файлов в хранилище проекта: загрузка с предварительной проверкой на актуальность


,Имя вставляемого файла,Директория вставляемого файла,Минимальная дата во вставляемом файле,Максимальная дата во вставляемом файле,Минимальное время звонка во вставляемом файле,Максимальное время звонка во вставляемом файле,Имя обновляемого/добавляемого файла,Директория обновляемого/добавляемого файла,Минимальное время звонка в обновляемом/добавляемом файле,Максимальное время звонка в обновляемом/добавляемом файле,Статус
0,Отчет по вызовам домена 997267.34.rt.ru (1).xlsx,C:\Users\AdminOS\Downloads,2022-12-17,2022-12-17,08:01:10,19:46:53,2022-12-18.xlsx,.\Data source\ВМК,08:01:10,19:46:53,Добавление файла выполнено!
1,Отчет по вызовам домена 997267.34.rt.ru (2).xlsx,C:\Users\AdminOS\Downloads,2022-12-18,2022-12-18,08:01:25,19:59:31,2022-12-19.xlsx,.\Data source\ВМК,08:01:25,19:59:31,Добавление файла выполнено!
2,Отчет по вызовам домена 997267.34.rt.ru.xlsx,C:\Users\AdminOS\Downloads,2022-12-16,2022-12-16,08:01:28,19:55:18,2022-12-17.xlsx,.\Data source\ВМК,08:01:28,19:55:18,Добавление файла выполнено!
3,Отчет по вызовам домена 553009.34.rt.ru (1).xlsx,C:\Users\AdminOS\Downloads,2022-12-17,2022-12-17,08:00:46,19:59:08,2022-12-18.xlsx,.\Data source\ВМК2,08:00:46,19:59:08,Добавление файла выполнено!
4,Отчет по вызовам домена 553009.34.rt.ru (2).xlsx,C:\Users\AdminOS\Downloads,2022-12-18,2022-12-18,08:00:34,19:45:44,2022-12-19.xlsx,.\Data source\ВМК2,08:00:34,19:45:44,Добавление файла выполнено!
5,Отчет по вызовам домена 553009.34.rt.ru.xlsx,C:\Users\AdminOS\Downloads,2022-12-16,2022-12-16,08:00:40,19:55:46,2022-12-17.xlsx,.\Data source\ВМК2,08:00:40,19:55:46,Добавление файла выполнено!
6,Отчет по вызовам в домене 997267.34.rt.ru (1).xlsx,C:\Users\AdminOS\Downloads,2022-12-17,2022-12-17,00:10:20,23:49:04,2022-12-17.xlsx,.\Data source\Ростелеком,00:10:20,23:49:04,Добавление файла выполнено!
7,Отчет по вызовам в домене 997267.34.rt.ru (2).xlsx,C:\Users\AdminOS\Downloads,2022-12-18,2022-12-18,00:01:33,20:05:27,2022-12-18.xlsx,.\Data source\Ростелеком,00:01:33,20:05:27,Добавление файла выполнено!
8,Отчет по вызовам в домене 997267.34.rt.ru.xlsx,C:\Users\AdminOS\Downloads,2022-12-16,2022-12-16,00:31:23,23:42:17,2022-12-16.xlsx,.\Data source\Ростелеком,00:31:23,23:42:17,Добавление файла выполнено!
